**36. La 3-upla de palabras más común en los comentarios de los logs (⭐⭐)**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=48a7ce4af6825bfb0dbca6fdfd2325a403fed771f26e472894ed5c0ef9001262
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InReleas

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [6]:
sqlContext = SQLContext(sc)
df_logs = sqlContext.read.csv('/content/drive/MyDrive/datosTP1/logs.csv', header=True, inferSchema=True)
rdd_logs = df_logs.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
rdd_logs.take(5)

[Row(item_id='1', timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id='2', timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id='3', timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id='4', timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id='7856', contributor_ip=None, comment='Sin información de orige

In [8]:
comments_validos = rdd_logs.filter(lambda x: x.comment != None).map(lambda x: x.comment)

In [9]:
comments_validos.take(5)

["el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'",
 "el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'",
 "el contenido era: '#REDIRECT [[Flash]]'",
 'Sin información de origen ni de licencia. Hay alternativas en commons',
 'Sustituida por  [[commons:Image:Heckert_GNU.png]]']

Defino un patrón para quedarme solo con conjuntos de 2 o más caracteres.

In [10]:
import re
patron = re.compile(r"\b\w{2,}\b")

Uso map para obtener un rdd que dentro tenga los tokens para cada comentario pero separados entre sí. Así poder diferenciar correctamente las 3-uplas que son de comentarios diferentes luego.

In [11]:
tokens = comments_validos.map(lambda x: re.findall(patron, x.lower()) )

Separo cada token en 3-uplas y creo tuplas con las 3-uplas como "clave".

In [12]:
tres_uplas = tokens.flatMap(lambda x: ngrams(x,3) ).map(lambda x: (x,1)).cache()   

In [14]:
tres_uplas.take(10)

[(('el', 'contenido', 'era'), 1),
 (('contenido', 'era', 'si'), 1),
 (('era', 'si', 'aprendés'), 1),
 (('si', 'aprendés', 'pensamiento'), 1),
 (('aprendés', 'pensamiento', 'científico'), 1),
 (('pensamiento', 'científico', 'te'), 1),
 (('científico', 'te', 'va'), 1),
 (('te', 'va', 'aenterar'), 1),
 (('el', 'contenido', 'era'), 1),
 (('contenido', 'era', 'destruir'), 1)]

In [15]:
tres_uplas.reduceByKey(lambda x,y: x + y).reduce(lambda x,y: x if x[1] > y[1] else y)

(('el', 'contenido', 'era'), 1134440)